In [ ]:
#Install dependencies
!pip install qiskit

In [3]:
#import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [10]:
#read csv
df = pd.read_csv("/content/image_data_final_quantum (2).csv")

In [13]:
from sklearn.decomposition import PCA

# Load the CSV file
df = pd.read_csv("/content/image_data_final_quantum (2).csv")
d = df
X = d.drop(['class'], axis=1)

# Apply PCA to extract 10 features
pca = PCA(n_components=10)
X = pca.fit_transform(X)

# Target variable
y = d['class']

In [14]:
from sklearn.model_selection import train_test_split
from qiskit.utils import algorithm_globals

#train test split
algorithm_globals.random_seed = 123
train_features, test_features, train_labels, test_labels = train_test_split( X, y, train_size=0.7, random_state=algorithm_globals.random_seed)

In [16]:
#encode our data as qubits by creating the quantum circuit
from qiskit.circuit.library import ZZFeatureMap
num_features = train_features.shape[1]
feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1)
# feature_map.decompose().draw(output='mpl', fold=20, filename='featuremap' )

In [18]:
from qiskit.circuit. library import RealAmplitudes

ansatz = RealAmplitudes (num_qubits=num_features, reps=3)
ansatz.measure_all()

In [19]:
# combine our feature map with our circuit
circuit=feature_map.compose(ansatz)
initial_point = np.random.random(ansatz.num_parameters)
initial_point

In [21]:
#Encoding the input features
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
train_labels = train_labels.to_numpy()
test_labels = test_labels.to_numpy()
train_labels_oh = encoder.fit_transform(train_labels.reshape(-1, 1)).toarray()
test_labels_oh = encoder.fit_transform(test_labels.reshape(-1, 1)).toarray()

In [ ]:
#Create a VQC object for the QASM simulator
from qiskit import BasicAer, execute
from qiskit.algorithms.optimizers import SPSA
from qiskit_machine_learning.algorithms.classifiers import VQC
vqc = VQC(feature_map=feature_map,ansatz=ansatz,loss='cross_entropy',optimizer=SPSA( ),initial_point=initial_point,quantum_instance=BasicAer.get_backend('qasm_simulator'))

In [ ]:
# Import time module
import time
 # record start time
start = time.time()

r = vqc.fit(train_features,train_labels_oh)

end = time.time()

print("The time of execution of above program is :",
      (((end-start) * 10**3)/1000)/60, "mins")

In [ ]:
#Prediction
predicted_labels_test = vqc.predict(test_features)
predicted_labels_train = vqc.predict(train_features)

In [ ]:
from sklearn.metrics import accuracy_score
print(f'Quantum VQC (sklearn) test accuracy {accuracy_score(test_labels,predicted_labels_test):}')
print(f'Quantum VQC (sklearn) train accuracy {accuracy_score(train_labels,predicted_labels_train):}')